In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df18 = df18.apply(pd.to_numeric, errors = 'coerce')
df18.describe().iloc[:, :14]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal
count,1.070040e+05,106782.000000,106782.000000,106560.000000,103230.000000,107004.000000,106671.000000,106560.000000,106560.000000,103563.000000,103563.000000,0.0,106449.000000,106782.000000
mean,1.368933e+18,79.417879,58.929314,69.475000,1.717204,51.494813,59.060354,5.034375,9.509375,610.946409,1843.022508,NaN,0.003754,0.064392
std,2.599832e+16,16.616016,16.623184,16.289049,7.631510,15.933838,13.760407,8.623523,9.758051,73.302920,75.744141,NaN,0.067901,0.257434
min,1.325376e+18,24.000000,15.000000,21.000000,-31.000000,3.000000,19.000000,0.000000,0.000000,519.000000,1721.000000,NaN,0.000000,0.000000
25%,1.346177e+18,69.000000,45.000000,57.000000,-3.000000,39.000000,48.000000,0.000000,0.000000,538.000000,1760.000000,NaN,0.000000,0.000000
50%,1.368187e+18,82.000000,62.000000,72.000000,3.000000,56.000000,63.000000,0.000000,7.000000,613.000000,1842.000000,NaN,0.000000,0.000000
75%,1.391494e+18,93.000000,74.000000,83.000000,7.000000,66.000000,72.000000,8.000000,18.000000,657.000000,1922.000000,NaN,0.000000,0.005000
max,1.414714e+18,108.000000,84.000000,95.000000,27.000000,72.000000,77.000000,44.000000,30.000000,731.000000,1940.000000,NaN,2.000000,3.060000


In [3]:
df18['store_nbr'].unique()

array([36], dtype=int64)

In [6]:
df18_drop_columns = ['date', 'codesum', 'station_nbr', 'store_nbr']
df18 = df18.drop(columns = df18_drop_columns)

In [7]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df18_columns = df18.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df18_columns:
    if (i == 'resultdir'):
        df18[i].fillna(df18[i].mode()[0], inplace=True)
        print(df18[i].mode()[0])
    else:
        df18[i].fillna(df18[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df18['relative_humility'] = 100*(np.exp((18.625*((df18['dewpoint']-32)/1.8))/(243.04+((df18['dewpoint']-32)/1.8)))/np.exp((18.625*((df18['tavg']-32)/1.8))/(243.04+((df18['tavg']-32)/1.8))))

# 체감온도 계산
df18["windchill"] = 35.74 + 0.6218*df18["tavg"] - 35.75*(df18["avgspeed"]**0.18) + 0.4275*df18["tavg"]*(df18["avgspeed"]**0.18)

df18 = df18[df18['units'] != 0]

17


In [8]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df18 = sm.OLS.from_formula('np.log1p(units) ~ depart + sunrise + sunset + tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df18)
result_df18 = model_df18.fit()

print(result_df18.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     854.5
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:59:25   Log-Likelihood:                -1811.3
No. Observations:                2836   AIC:                             3681.
Df Residuals:                    2807   BIC:                             3853.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]         0.0117      0.02

In [9]:
model_df18 = sm.OLS.from_formula('np.log1p(units) ~ scale(depart) + scale(sunset) + scale(sunrise) + scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df18)
result_df18 = model_df18.fit()

print(result_df18.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     854.5
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:59:52   Log-Likelihood:                -1811.3
No. Observations:                2836   AIC:                             3681.
Df Residuals:                    2807   BIC:                             3853.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [10]:
anova_result_df18 = sm.stats.anova_lm(result_df18).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df18[anova_result_df18['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(tmin),1.0,0.878244,0.878244,4.138941,4.200121e-02
C(other),1.0,0.896123,0.896123,4.223197,3.996731e-02
scale(dewpoint),1.0,1.310003,1.310003,6.173706,1.302413e-02
scale(resultspeed),1.0,1.615870,1.615870,7.615183,5.825455e-03
scale(depart),1.0,2.122708,2.122708,10.003778,1.578878e-03
scale(relative_humility),1.0,3.526154,3.526154,16.617858,4.698806e-05
scale(heat),1.0,3.683469,3.683469,17.359243,3.187327e-05
scale(sunrise),1.0,4.830689,4.830689,22.765798,1.923537e-06
C(week7),6.0,19.734633,3.289105,15.500710,1.435262e-17
C(item_nbr),3.0,5034.859553,1678.286518,7909.333610,0.000000e+00


In [11]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df18.values, i) for i in range(df18.shape[1])]
vif["features"] = df18.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(other) + scale(depart) + scale(week7) + C(item_nbr) 

,VIF Factor,features
0,1.108664,snowfall
1,1.233994,resultdir
2,1.246557,item_nbr
3,1.260080,units
4,1.306676,preciptotal
5,2.201140,other
6,2.812208,week7
7,2.815697,weekend
8,2.830204,depart
9,6.995299,sunset


In [12]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df18 = sm.OLS.from_formula('np.log1p(units) ~ C(other) + scale(depart) + scale(week7) + C(item_nbr) + 0', data = df18)
result_df18 = model_df18.fit()

print(result_df18.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     3743.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:01:32   Log-Likelihood:                -1901.2
No. Observations:                2836   AIC:                             3816.
Df Residuals:                    2829   BIC:                             3858.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(other)[0]           1.1771      0.02

In [13]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X18 = df18[['other', 'depart', 'week7', 'item_nbr']]
y18 = df18['units']
model18 = LinearRegression()

cv18 = KFold(n_splits=18, shuffle=True, random_state=0)

cross_val_score(model18, X18, y18, scoring="r2", cv=cv18)

array([0.12608266, 0.16313887, 0.20498937, 0.16297503, 0.11910576,
       0.12663307, 0.21127656, 0.14861174, 0.17471466, 0.29227833,
       0.15815256, 0.22850942, 0.16426886, 0.24513417, 0.10403061,
       0.21666954, 0.15778536, 0.20164158])